In [1]:
!pip install -q --upgrade datasets transformers accelerate evaluate  # Upgrade to latest
!pip install -q huggingface_hub  # For saving to HF Hub

In [2]:
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"  # Ignore deprecation; disables WandB
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # TEMP: Sync CUDA for exact error if persists (remove after fix)

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate
import torch
from huggingface_hub import notebook_login

notebook_login()  # For HF push

In [3]:
# Load dataset
dataset = load_dataset("cardiffnlp/x_sensitive")

# Define label keys
label_keys = ['conflictual', 'drugs', 'profanity', 'selfharm', 'sex', 'spam']

# FIXED: Collapse to binary INT label (0/1 for CE loss with num_labels=2)
def collapse_to_binary(example):
    binary_label = 1 if any(example[key] == 1 for key in label_keys) else 0
    example['labels'] = int(binary_label)  # INT: For CrossEntropyLoss (long tensors)
    return example

# Apply to all splits
dataset = dataset.map(collapse_to_binary)

# Quick stats
print("Binary label distribution:")
for split in dataset:
    labels = dataset[split]['labels']
    print(f"{split}: Sensitive {np.sum(labels)} / {len(labels)} ({np.mean(labels)*100:.1f}%)")

# Tokenizer
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=False, max_length=128)

# Tokenize splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text", "keyword"] + label_keys + [k for k in dataset['train'].features if '_highlight' in k] + ['#labels'])
tokenized_datasets.set_format("torch")

# Verify: Print a sample (labels should be int/long)
print("\nSample after tokenization:")
sample = tokenized_datasets["train"][0]
print(f"Keys: {list(sample.keys())}")
print(f"Labels type/value: {type(sample['labels'])} / {sample['labels']}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Binary label distribution:
train: Sensitive 2381 / 5000 (47.6%)
validation: Sensitive 436 / 1000 (43.6%)
test: Sensitive 1082 / 2000 (54.1%)


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


Sample after tokenization:
Keys: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
Labels type/value: <class 'torch.Tensor'> / 0


In [4]:
# FIXED: Load model for binary (2 classes: 0=not, 1=sensitive; CE loss)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # 2 classes for CrossEntropyLoss
    problem_type="single_label_classification"
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Metrics (argmax for 2-class preds)
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # FIXED: Argmax for CE (no sigmoid)
    return {
        'accuracy': accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        'f1': f1.compute(predictions=predictions, references=labels, average='binary')['f1']
    }

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
training_args = TrainingArguments(
    output_dir="./x-sensitive-deberta-binary",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None,  # No logging
    push_to_hub=True,
    hub_model_id="faketut/x-sensitive-deberta-binary",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,  # No deprecation warning
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start fine-tuning
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.342743,0.868000,0.839416
2,0.438900,0.313275,0.880000,0.854369
3,0.438900,0.360850,0.869000,0.853958


TrainOutput(global_step=939, training_loss=0.37009377119005427, metrics={'train_runtime': 480.74, 'train_samples_per_second': 31.202, 'train_steps_per_second': 1.953, 'total_flos': 601684107268704.0, 'train_loss': 0.37009377119005427, 'epoch': 3.0})

In [6]:
# Evaluate on test set
test_results = trainer.evaluate(tokenized_datasets["test"])
print("Test Results:", test_results)

# Save final checkpoint locally and to HF Hub
trainer.save_model("./x-sensitive-deberta-binary-final")
# Already pushed via push_to_hub=True; or manual: trainer.push_to_hub()

Test Results: {'eval_loss': 0.4083416759967804, 'eval_accuracy': 0.8265, 'eval_f1': 0.8288110508140109, 'eval_runtime': 14.5812, 'eval_samples_per_second': 137.163, 'eval_steps_per_second': 8.573, 'epoch': 3.0}


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...-binary/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...798548.4746f84f0bd9.960.4: 100%|##########| 5.39kB / 5.39kB            

...

  ...798300.4746f84f0bd9.960.2: 100%|##########| 5.40kB / 5.40kB            

  ...9396.4746f84f0bd9.12355.0: 100%|##########| 6.98kB / 6.98kB            

  ...798654.4746f84f0bd9.960.5: 100%|##########| 5.39kB / 5.39kB            

  ...-deberta-binary/spm.model: 100%|##########| 2.46MB / 2.46MB            

  ...-binary/model.safetensors:   3%|3         | 25.1MB /  738MB            

In [7]:
from transformers import pipeline
classifier = pipeline("text-classification", model="./x-sensitive-deberta-binary-final", return_all_scores=True)
print(classifier("This is so fucking cool!"))  # [{'label': 'LABEL_1', 'score': 0.95}] -> sensitive

Device set to use cuda:0


[[{'label': 'LABEL_0', 'score': 0.01475132629275322}, {'label': 'LABEL_1', 'score': 0.9852486252784729}]]


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
